<a href="https://colab.research.google.com/github/shamil-k/Salary-And-Satisfaction-Prediction-Using_RandomForest/blob/main/Salary_Satisf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Business Understanding 

You have to clean this data, prepare it for ML models, build more features and ultimately build a random forest (scikit).
Split the dataset into 80:20, train:test. 
Print the Weighted F1 on test dataset (target_satisfaction) and RMSE (Target_salary)
 


# some of my references:



https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

https://statistics.berkeley.edu/sites/default/files/tech-reports/666.pdf

https://github.com/shamil-k/Feature-Engineer-All-You-need


https://towardsdatascience.com/getting-deeper-into-categorical-encodings-for-machine-learning-2312acd347c8

https://github.com/shamil-k/Feature-Engineer-All-You-need
                https://heartbeat.fritz.ai/
              


https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/
               
               
https://contrib.scikit-learn.org/category_encoders/hashing.html

# Data Mining

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import sklearn
!pip install category_encoders


**Reading The Data Set Using Pandas.**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/New folder/salary,satisfaction - salary,satisfaction.csv', parse_dates=True, dayfirst=True, index_col=0)
df.head()

# Data Cleaning/Preprocessing  

In [ ]:

df.isnull().sum()


In [ ]:
df.dtypes

In [ ]:

df['FormalEducation'] = df['FormalEducation'].fillna('Not-Specified')
df['UndergradMajor'] = df['UndergradMajor'].fillna('Not-Specified')
df['DevType'] = df['DevType'].fillna('Not-Specified')
df['HopeFiveYears'] = df['HopeFiveYears'].fillna('Not-Specified')
df['YearsCoding'] = df['YearsCoding'].fillna('0-2')
df['YearsCodingProf'] = df['YearsCodingProf'].fillna('0-2')
df['JobSearchStatus'] = df['JobSearchStatus'].fillna('Not-Specified')
df['LastNewJob'] = df['LastNewJob'].fillna('Not-Specified')
df['EducationTypes'] = df['EducationTypes'].fillna('Not-Specified')
df['AgreeDisagree1'] = df['AgreeDisagree1'].fillna('Not-Specified')
df['Age'] = df['Age'].fillna('Not-Specified')
df['emp_length'] = df['emp_length'].fillna('< 1 year')
df['Age'] = df['Age'].fillna('Not-Specified')
df['title'] = df['title'].fillna('Not-Specified')

In [ ]:
df.isnull().sum()

In [ ]:
chars_to_remove = ['years', 'or more years', 'years old', '$', '<']
regular_expression = '[' + re.escape (''. join (chars_to_remove)) + ']'
df['YearsCodingProf'] = df['YearsCodingProf'].str.replace(regular_expression, '', regex=True) 
df['title'] = df['title'].str.replace(regular_expression, '', regex = True)
df['LOAN_AMT'] = df['LOAN_AMT'].str.replace(regular_expression, '', regex = True)
df['emp_length'] = df['emp_length'].str.replace(regular_expression, '', regex = True)

In [ ]:
df  = df.reset_index()
df = df.drop(['id', 'Time', 'EducationTypes',  'latitude', 'longitude', 'YearsCoding' ], axis=1)


In [ ]:
df.head()

In [ ]:
chars_to_remove = ['years', 'or more years', 'years old', '$']
regular_expression = '[' + re.escape (''. join (chars_to_remove)) + ']'
df['YearsCodingProf'] = df['YearsCodingProf'].str.replace(regular_expression, '', regex=True) 
df['title'] = df['title'].str.replace(regular_expression, '', regex = True)
df['LOAN_AMT'] = df['LOAN_AMT'].str.replace(regular_expression, '', regex = True)

In [ ]:
df = df.drop('dateAdded', axis=1)

## Encoding Categorical Data 

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import category_encoders as ce

In [ ]:
df.apply(lambda x: len(x.unique()))

In [ ]:
df.dtypes

In [ ]:
encoder = ce.TargetEncoder(cols=['Business Title'])
encoder

In [ ]:
df['Business Title'] = encoder.fit_transform(df['Business Title'],df['Target_Salary'])


In [ ]:
encoder = ce.TargetEncoder(cols=['FormalEducation'])
encoder
df['FormalEducation'] = encoder.fit_transform(df['FormalEducation'],df['Target_Salary'])

**Target Guided Ordinal Encoding** 
Ordering the labels according to the target
Replace the labels by the joint probability of being 1 or 0 

In [ ]:
df.apply(lambda x: len(x.unique()))

In [ ]:
df.emp_length.unique()

df.groupby(['emp_length'])['Target_Satisfied'].mean().sort_values().index

In [ ]:
ordinal_labels=df.groupby(['emp_length'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels

In [ ]:
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
ordinal_labels2

Mean Encoding

In [ ]:
mean_ordinal=df.groupby(['emp_length'])['Target_Satisfied'].mean().to_dict()
mean_ordinal

In [ ]:
df['emp_length']=df['emp_length'].map(mean_ordinal)

In [ ]:
# For LOAN_AMT column
df.LOAN_AMT.unique()
df.groupby(['LOAN_AMT'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels=df.groupby(['LOAN_AMT'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['LOAN_AMT'])['Target_Satisfied'].mean().to_dict()
df['LOAN_AMT']= df['LOAN_AMT'].map(mean_ordinal)





# For Civil Service Title column
df['Civil Service Title'].unique()
ordinal_labels=df.groupby(['Civil Service Title'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['Civil Service Title'])['Target_Satisfied'].mean().to_dict()
df['Civil Service Title']= df['Civil Service Title'].map(mean_ordinal)





# For Division/Work Unit Title column
df['Division/Work Unit'].unique()
ordinal_labels=df.groupby(['Division/Work Unit'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['Division/Work Unit'])['Target_Satisfied'].mean().to_dict()
df['Division/Work Unit']= df['Division/Work Unit'].map(mean_ordinal)


# For DevType column
df['DevType'].unique()
ordinal_labels=df.groupby(['DevType'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['DevType'])['Target_Satisfied'].mean().to_dict()
df['DevType']= df['DevType'].map(mean_ordinal)

# For earliest_cr_line column
df['earliest_cr_line'].unique()
ordinal_labels=df.groupby(['earliest_cr_line'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['earliest_cr_line'])['Target_Satisfied'].mean().to_dict()
df['earliest_cr_line']= df['earliest_cr_line'].map(mean_ordinal)

# For HopeFiveYears column
df['HopeFiveYears'].unique()
ordinal_labels=df.groupby(['HopeFiveYears'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['HopeFiveYears'])['Target_Satisfied'].mean().to_dict()
df['HopeFiveYears']= df['HopeFiveYears'].map(mean_ordinal)




# For Age column
df['Age'].unique()
ordinal_labels=df.groupby(['Age'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['Age'])['Target_Satisfied'].mean().to_dict()
df['Age']= df['Age'].map(mean_ordinal)

# For purpose column
df['purpose'].unique()
ordinal_labels=df.groupby(['purpose'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['purpose'])['Target_Satisfied'].mean().to_dict()
df['purpose']= df['purpose'].map(mean_ordinal)


# For Title column
df['title'].unique()
ordinal_labels=df.groupby(['title'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['title'])['Target_Satisfied'].mean().to_dict()
df['title']= df['title'].map(mean_ordinal)



# For last_pymnt_d column
df['last_pymnt_d'].unique()
ordinal_labels=df.groupby(['last_pymnt_d'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['last_pymnt_d'])['Target_Satisfied'].mean().to_dict()
df['last_pymnt_d']= df['last_pymnt_d'].map(mean_ordinal)



# For last_credit_pull_d column
df['last_credit_pull_d'].unique()
ordinal_labels=df.groupby(['last_credit_pull_d'])['Target_Satisfied'].mean().sort_values().index
ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
mean_ordinal=df.groupby(['last_credit_pull_d'])['Target_Satisfied'].mean().to_dict()
df['last_credit_pull_d']= df['last_credit_pull_d'].map(mean_ordinal)

**Binary Encoding**
Binary encoding is a combination of Hash encoding and one-hot encoding. In this encoding scheme, the categorical feature is first converted into numerical using an ordinal encoder. Then the numbers are transformed in the binary number. After that binary value is split into different columns.Here We are using Binary Encoding for column that have less number of labels 

Binary encoding is a memory-efficient encoding scheme as it uses fewer features than one-hot encoding. Further, It reduces the curse of dimensionality for data with high cardinality.

In [ ]:
encoder= ce.BinaryEncoder(cols=['UndergradMajor'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['UndergradMajor']) 
df2 = data_encoded
df_row = pd.concat([df, df2], axis=1)


encoder= ce.BinaryEncoder(cols=['CompanySize'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['CompanySize']) 
df4 = data_encoded
df_row = pd.concat([df_row, df4], axis=1)


encoder= ce.BinaryEncoder(cols=['YearsCodingProf'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['YearsCodingProf']) 
df5 = data_encoded
df_row = pd.concat([df_row, df5], axis=1)



encoder= ce.BinaryEncoder(cols=['YearsCodingProf'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['YearsCodingProf']) 
df5 = data_encoded
df_row = pd.concat([df_row, df5], axis=1)

encoder= ce.BinaryEncoder(cols=['JobSearchStatus'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['JobSearchStatus']) 
df6 = data_encoded
df_row = pd.concat([df_row, df6], axis=1)


encoder= ce.BinaryEncoder(cols=['LastNewJob'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['LastNewJob']) 
df7 = data_encoded
df_row = pd.concat([df_row, df7], axis=1)


encoder= ce.BinaryEncoder(cols=['AgreeDisagree1'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['AgreeDisagree1']) 
df8 = data_encoded
df_row = pd.concat([df_row, df8], axis=1)


encoder= ce.BinaryEncoder(cols=['AgreeDisagree1'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['AgreeDisagree1']) 
df8 = data_encoded
df_row = pd.concat([df_row, df8], axis=1)


encoder= ce.BinaryEncoder(cols=['term'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['term']) 
df9 = data_encoded
df_row = pd.concat([df_row, df9], axis=1)

encoder= ce.BinaryEncoder(cols=['loan_status'],return_df=True)
#Fit and Transform Data 
data_encoded=encoder.fit_transform(df['loan_status']) 
df10 = data_encoded
df_row = pd.concat([df_row, df10], axis=1)




In [ ]:
df_cleaned = df_row.drop(['UndergradMajor', 'CompanySize', 'YearsCodingProf',
             'JobSearchStatus', 'LastNewJob', 'AgreeDisagree1', 'term','loan_status'], axis=1)

In [ ]:
# checking the correlation 


df.corr()


In [ ]:
# Here the last_pymnt_amnt column have high -ve correlation 
# dropping the -ve correlate feature
df_cleaned.drop('last_pymnt_amnt', axis=1)

In [ ]:
df_cleaned.dtypes

## Dealing with Imbalanced Data


In [ ]:
target_satisfied = df[df['Target_Satisfied'] == 1]
target_not_satisfied = df[df['Target_Satisfied'] == 0]

count_classes = pd.value_counts(df['Target_Satisfied'], sort = True)

count_classes.plot(kind = 'bar', rot=0)

plt.title("Transaction Class Distribution")
LABELS = ['Target Satisfied', 'Target_Not_Satisfied']
plt.xticks(range(2), LABELS)

plt.xlabel("Class")

plt.ylabel("Frequency")


**OverSampling**

I observed that the data is not in-balanced so we are doing over sampling instead of under sampling because the records are not huge.

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# shape Before over_sampling
print('Before Over_Sampling', target_satisfied.shape, target_not_satisfied.shape)

# Splitting Dependent And Independent Features
X = df_cleaned.drop('Target_Satisfied', axis=1)
y = df_cleaned['Target_Satisfied']


os =  RandomOverSampler(ratio=0.5)
X_train_res, y_train_res = os.fit_sample(X, y)

# shape After over_sampling
print('After Over_Sampling', X_train_res.shape,y_train_res.shape)

In [ ]:
# Counter will help to count labels
# riginal dataset shape Counter
# Resampled dataset shape Counter
from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

# Modeling

In [ ]:

from sklearn.model_selection import train_test_split
X = X_train_res
y = y_train_res


# Randomly splitting train-test data (80,20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# Building Classification Model To Predict Satisfied/Not-Satisfied

Here I used RandomForestClassier As Per The Requirements  For Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_train, y_train)

# Model Evaluating



**WEIGHTED F1 SCORE: 0.926605504587156**

In [ ]:
# performing predictions on the test dataset
y_pred = clf.predict(X_test)

# metrics are used to find accuracy or error
from sklearn import metrics  
print()


from sklearn.metrics import f1_score
print('WEIGHTED F1 SCORE :', f1_score(y_test,y_pred))

# Confusion Matrix
print(metrics.confusion_matrix(y_test, y_pred))


Model Deployment

In [ ]:

import joblib
# saving the dataframe
df_cleaned.to_csv('After_Preprocessing.csv')

# save the model to disk here iam saving my drive. 
# also shared the model and data after preprocessing on git
model = clf
filename = '/content/drive/MyDrive/New folder/CLF_Model.pkl'
joblib.dump(model, filename)

In [ ]:
# Load the model from the file
# RFC_from_joblib = joblib.load('/content/drive/MyDrive/New folder/CLF_Model.pkl') 
  
# # Use the loaded model to make predictions
# y_pred = RFC_from_joblib.predict(X_test)
  
# # using metrics module for accuracy calculation
# print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))
# print('', f1_score(y_test,y_pred))
# print(metrics.confusion_matrix(y_test, y_pred))

# Building a Regression Model To Predict Target Salary

In [ ]:

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print('Mean_Squered_Error', mse)
r2 = r2_score(y_test, y_pred)
print('R^2 SCORE:',r2)


rmse = sqrt(mean_squared_error(y_test, y_pred))
print('ROOT MEAN SQUERE ERROR: ',rmse)

**ROOT MEAN SQUERE ERROR:0.2225828457207991**